In [1]:
import flymovie as fm
import numpy as np
import scipy.ndimage as ndi
from importlib import reload
from flymovie.general_functions import extract_box, stack_normalize_minmax

%load_ext autoreload
%autoreload 2

In [2]:
stack1 = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-100x50-field4-1AU-green-01.czi', swapaxes=False)
stack2 = fm.read_czi('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/psf/2022-02-09/tetraspeck-100x50-field5-1AU-green-01.czi', swapaxes=False)

stacklist = [stack1[0], stack1[1], stack2[0], stack2[1], stack2[2], stack2[3], stack2[4]]

In [3]:
boxes = fm.extract_beads_batch(stacklist, 20, (41, 25, 25))

In [4]:
fm.viewer([boxes.max(axis=(1)) * 1000, boxes.max(axis=(2)) * 1000], 4)

interactive(children=(Dropdown(description='Color', index=2, options=('gray_r', 'Gators', 'cividis', 'Greens',…

In [5]:
bad_beads = [0,29, 58, 60, 84]

In [6]:
boxes_good = fm.remove_bad_beads(boxes, bad_beads)
psf = boxes_good.mean(axis=0)
fm.viewer(psf * 1000, 5)

interactive(children=(Dropdown(description='Color', index=2, options=('gray_r', 'Gators', 'cividis', 'Greens',…

In [8]:
fm.viewer([psf.max(axis=0) * 1000, psf.max(axis=1) * 1000, psf.max(axis=2) * 1000], 5)

interactive(children=(Dropdown(description='Color', index=5, options=('Gators', 'Reds', 'inferno', 'viridis', …

### Save original

In [ ]:
psf_file = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220503_100nmx50x50_41pxx25x25.pkl'
fm.save_pickle(psf, psf_file)

### Stretch and save

In [9]:
stretch_factor = 3
stretched = ndi.zoom(psf, (stretch_factor, 1, 1))
# Squeeze a little if desired to reduce kernel size.
stretched = stretched[22:-22, 2:-2, 2:-2]
stretched = stretched / np.max(stretched)
fm.viewer([psf.max(axis=1) * 1000, stretched.max(axis=1) * 1000], 5)

interactive(children=(Dropdown(description='Color', index=2, options=('gray_r', 'Gators', 'cividis', 'Greens',…

In [10]:
stretched.shape

(79, 21, 21)

In [11]:
stretched_file = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220503_100nmx50x50_79pxx21x21_zstretch3.0.pkl'
fm.save_pickle(stretched, stretched_file)

### Fit and stretch

In [ ]:
def make_kernel(x, shape):
    """Make kernel of certain shape from fitted 3d gaussian function."""
    x1 = list(x.copy())
    x1[0] = int(shape[0] / 2)
    x1[1] = int(shape[1] / 2)
    x1[2] = int(shape[2] / 2)
    g = fm.gaussian3d(*x1)
    return g(*np.indices(shape))

In [ ]:
shape = (51,20,20)
opt = fm.fitgaussian3d(psf / np.max(psf))
x = opt.x.copy()
kernel_f = make_kernel(x, shape)

In [ ]:
fm.viewer(kernel_f.max(axis=1) * 1000, 5)

In [ ]:
# See current z-width.
original_zwidth = x[4]
print(original_zwidth)

In [ ]:
stretch_factor = 1.5
x[4] = stretch_factor * original_zwidth
shape = (51,20,20)
fitted = make_kernel(x, shape)
fm.viewer(fitted.max(axis=1) * 1000, 5)

In [ ]:
fitted_file = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/fitted_20220503_z4.2_100nmx50x50_51pxx20x20.pkl'
fm.save_pickle(fitted, fitted_file)